In [1]:
import numpy as np
import tensorflow as tf

tf.__version__

'1.14.0'

In [2]:
cpus = tf.config.experimental.list_physical_devices('CPU') # get a list of cpus
try:
    tf.config.experimental.set_virtual_device_configuration(cpus[0],
                                                            [tf.config.experimental.VirtualDeviceConfiguration(),
                                                             tf.config.experimental.VirtualDeviceConfiguration(),
                                                             tf.config.experimental.VirtualDeviceConfiguration()])
    logical_cpus = tf.config.experimental.list_logical_devices('CPU')
except RuntimeError as e:
    print(e)

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/cpu:0", "/cpu:1", "/cpu:2"])

W1229 22:20:47.994878 17380 cross_device_ops.py:1177] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [4]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[]

In [5]:
def create_model():
    inputs = tf.keras.layers.Input(shape=(5,))
    x = tf.keras.layers.Dense(10)(inputs)
    outputs = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.models.Model(inputs, outputs)

In [6]:
with mirrored_strategy.scope():
    model = create_model()

W1229 22:20:48.360223 17380 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[MirroredVariable:{
   0 /replica:0/task:0/device:CPU:0: <tf.Variable 'dense/kernel:0' shape=(5, 10) dtype=float32>,
   1 /replica:0/task:0/device:CPU:1: <tf.Variable 'dense/kernel/replica_1:0' shape=(5, 10) dtype=float32>,
   2 /replica:0/task:0/device:CPU:2: <tf.Variable 'dense/kernel/replica_2:0' shape=(5, 10) dtype=float32>
 }, MirroredVariable:{
   0 /replica:0/task:0/device:CPU:0: <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32>,
   1 /replica:0/task:0/device:CPU:1: <tf.Variable 'dense/bias/replica_1:0' shape=(10,) dtype=float32>,
   2 /replica:0/task:0/device:CPU:2: <tf.Variable 'dense/bias/replica_2:0' shape=(10,) dtype=float32>
 }, MirroredVariable:{
   0 /replica:0/task:0/device:CPU:0: <tf.Variable 'batch_normalization/gamma:0' shape=(10,) dtype=float32>,
   1 /replica:0/task:0/device:CPU:1: <tf.Variable 'batch_normalization/gamma/replica_1:0' shape=(10,) dtype=float32>,
   2 /replica:0/task:0/device:CPU:2: <tf.Variable 'batch_normalization/gamma/replica_2:0' shape=(10,

In [16]:
with mirrored_strategy.scope(): 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(x))

[[ 0.47601873  0.5158697  -0.47555354  0.32578492  0.26930517  0.08378947
  -0.24745312  0.5027873   0.3466404  -0.08635843]
 [ 0.21788245  0.04409963  0.2919168  -0.2278519  -0.03624141 -0.14428803
  -0.16353431  0.5316847  -0.61674076  0.10174274]
 [ 0.4714666  -0.56789744 -0.4356119  -0.32540238  0.09258407 -0.6309101
   0.10930359  0.5867911   0.3875031   0.57004505]
 [ 0.3023066   0.54046243  0.58091563 -0.3263338   0.6006467   0.48058528
  -0.087982   -0.2440919  -0.20643637 -0.39896756]
 [-0.23307902 -0.30479884  0.26334995 -0.12517506 -0.4524617   0.02994508
  -0.31943685  0.34426337 -0.10439253 -0.32729086]]


In [8]:
with mirrored_strategy.scope():
    print(model.get_updates_for(model.inputs))

[<tf.Tensor 'batch_normalization/packed:0' shape=(3, 10) dtype=float32>, <tf.Tensor 'batch_normalization/packed_1:0' shape=(3, 10) dtype=float32>]
